# playNano Demonstration Notebook

This notebook walks through the core functionality of **playNano**, a Python library for time-series AFM (Atomic Force Microscopy) data analysis. We will cover:

1. **Loading**: Importing AFM data into an `AFMImageStack`
2. **Processing**: Demonstration and visualisations of flattening and filtering operations
3. **Analysis**: Feature detection, tracking, and linking results into a tabular form
4. **Exporting**: Saving processed data and results using both I/O functions and analysis utilities

---

## 1. 🚀 Setup and Imports

Start by installing any dependencies and importing all required packages for AFM video analysis.

In [ ]:
# If playNano isn't installed ensure this notebook is opened from the
# repository root and uncomment the installation line.
# Install in editable mode if developing:
# !pip install -e .

from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib import animation

from playNano.afm_stack import AFMImageStack
from playNano.processing.pipeline import ProcessingPipeline
from playNano.analysis.pipeline import AnalysisPipeline

from playNano.io.export_data import (
    save_ome_tiff_stack,
    save_npz_bundle,
    save_h5_bundle,
)
from playNano.analysis.export import export_analysis_to_json
from playNano.analysis.utils.particles import (
    flatten_particle_features,
    plot_particle_labels_3d,
)




> **Tip**: Run each cell sequentially to see outputs inline.

---

## 2. 🗂️ Loading Data

Next, we’ll load our AFM dataset into an `AFMImageStack` so we can begin processing. `load_data()` builds a AFMImageStack object with the video frames as a 3D numpy array with dimensions height, width and frame index. The pixel to 'real' length scale conversion is read and stored in the `pixel_size_nm` attribute, scan rate is also read from the files and timestamps generated and stored in the `frame_metadata` attribute which contains an ordered list of the all the frames with metadata for each. 



In [ ]:
from pathlib import Path

# This block find the repo root so the test data can be accessed

def find_repo_root(marker=".git"):
    """Walk up parent directories until we find the repo root (marked by .git folder)."""
    path = Path.cwd()
    for parent in [path] + list(path.parents):
        if (parent / marker).exists():
            return parent
    raise FileNotFoundError("Could not find the repository root (no .git directory found).")

repo_root = find_repo_root()

# The demo path opens demo data from the test suit, to use your own data uncomment the your_path line.
# Then comment out the line that loads demo_path and uncomment the line that loads your_path. 

demo_path = repo_root / "tests" / "resources" / "sample_0.h5-jpk"
# your_path = Path(r"\path\to\your\data.h5-jpk")

channel = 'height_trace'  # common HS-AFM height channel

stack = AFMImageStack.load_data(demo_path, channel=channel)
# stack = AFMImageStack.load_data(your_path, channel="height_trace") # uncomment this to use your_path

print(f"Loaded {stack.n_frames} frames; each frame shape: {stack.image_shape}")
print(f"Pixel size: {stack.pixel_size_nm} nm")
print(stack.frame_metadata)




Inspect the first frame:



In [ ]:

frame0 = stack.get_frame(0)
meta0 = stack.get_frame_metadata(0)
print("Frame 0 metadata:", meta0)
plt.figure(figsize=(4,4))
plt.imshow(frame0, cmap="afmhot", origin="lower")
plt.title("Raw Frame 0")
plt.colorbar(label="Height (nm)")
plt.show()




---

## 3. 🔧 Processing Data

Now that the raw data is loaded, the frames can be processed and flattened using a reproducible pipeline for tilt correction, row alignmnent, and smoothing.

AFM data often has background tilt (scanner bow) and line-by-line hysteresis. We will apply a reproducible pipeline using the following playNano processing functions through the ProcessingPipeline class that record and applies all the processing steps:

1. **`remove_plane`**: Fits a 2D plane to each frame and subtracts it, removing large-scale tilt and scanner bow.
2. **`mask_mean_offset`**: Computes the mean height of each frame, masks detected features via a threshold on that mean, and subtracts a fraction (`factor=0.4`) of this offset to correct global drift while preserving particle signal.
3. **`row_median_align`**: Subtracts the median of unmasked pixels in each row, correcting line-by-line hysteresis and scan artefacts.
4. **`polynomial_flatten`**: Fits and subtracts a 2nd-order polynomial surface to each frame for higher-order background correction.
5. **`gaussian_filter`**: Applies a Gaussian filter with `sigma=1` to smooth residual noise.

Each of these functions includes detailed docstrings explaining parameters, mask usage, and behaviour.

### 3.1 🔄 Reproducible Processing Pipeline

The different filters and masks are added as steps in the pipeline using the `add_filter` and `add_mask` functions depending on whether the step is a filtering of masking operation. Some steps take parameters that can be specifed when add the step.

Once the pipeline is complete the process can be applied with the `apply` method, this applies the steps to the data, saving a copy or mask array after each step in either `stack.processed` or `stack.masks`. The origonal data is also retained in `stack.processed['raw']`.


In [ ]:

# Restore raw data
# Restore raw data if available
raw_data = stack.processed.get('raw')
if raw_data is not None:
    stack.data = raw_data


pipe = ProcessingPipeline(stack)
pipe.add_filter('remove_plane')
pipe.add_mask('mask_mean_offset', factor=1)
pipe.add_filter('row_median_align')
pipe.add_filter('polynomial_flatten', order=2)
pipe.add_filter('gaussian_filter', sigma=1.5)

proc_record = pipe.run()

# The processing steps and the associated parameters are stored
# within the 'providence' AFMImageStack attirbute.
# Acessing this provides are record of the processing steps used.
print("Processing steps executed with parameters:")
for step in stack.provenance['processing']['steps']:
    print(f" • {step['name']}: {step['params']}")


fig, ax = plt.subplots(figsize=(4,4))
im = ax.imshow(stack.data[0], cmap='afmhot', origin='lower')
cb = plt.colorbar(im, label="Height / nm")
plt.title("Processed Frame 0")
ax.axis('off')




### 3.2 🎞️ Animation of the Processed Stack


The process is applied to each frame of the AFM video.


In [ ]:

from IPython.display import HTML

def update(frame):
    global cb
    if cb:
        cb.remove()
        cb = None
    im.set_data(stack.data[frame])
    ax.set_title(f"Frame {frame}")
    return [im]


anim = animation.FuncAnimation(fig, update, frames=range(stack.n_frames), interval=100)
HTML(anim.to_jshtml())



### 3.3 🎬 Exporting as GIF

There is built in functions for the export of raw and processed data as GIF animations, these are annotated with a scale bar and a timestamp.


In [ ]:

from IPython.display import Image, display

from playNano.io.gif_export import export_gif

# Define output path and frame interval (ms)
out_path = Path('output') 
out_name = 'processed_stack'
# Write GIF from processed data
export_gif(
    afm_stack=stack,
    make_gif=True,
    output_folder=out_path,
    output_name=out_name,
    scale_bar_nm= 100,
)
print(f"GIF saved to {out_path}")


full_output_path = f"./{out_path}/{out_name}_filtered.gif"

display(Image(filename=full_output_path))





---

## 4. 🔍 Analysis

Once data is processed, it can be fed into an analysis pipeline with various analysis steps. Currently, there are built in modules to detect features (feature_detection and log_blob_detection) and then cluster or track these detections `particle_tracking`, `x_means_clustering`, `x_means_clustering` and `dbscan_clustering`).

In this example, `feature_detection` is used to identify particles using a threshold mask and then `particle_tracking` links these detections between frames. 


In [ ]:

# Clear any previous analysis
stack.analysis.clear()
stack.provenance['analysis']['steps'].clear()

# Initiallise an analysis pipeline 
analysis_pipeline = AnalysisPipeline()

# Add analysis steps:
analysis_pipeline.add(
    'feature_detection',
    mask_fn="mask_mean_offset",
    factor=1.15,
    min_size =100,
    fill_holes= True
)
analysis_pipeline.add('particle_tracking', max_distance=10)

# Run analysis
analysis_record = analysis_pipeline.run(stack)
print(
    "Analysis modules run:", [s['name'] for s in analysis_record['provenance']['steps']]
)




### 4.1 🕵️‍♂️ Feature Detection

Feature detection using the `feature_detection` module uses a threshold mask to identify features as mask, 'islands', which are then identified and filtered by size and location. Masks can either be generated during analysis, set by the argument `mask_fn`, or a mask previously generated during processing can be used by identifying the mask key in `previous_results` with `masks_key`.  

As well as detecting features, this module measures various features of the detected feature (min, max and mean values, centroid position and area etc.). The feature detection result dictionary contains a list of the features detected under `features_per_frame` with these metrics and others. 


In [ ]:

fd = stack.analysis['step_1_feature_detection']
total_feats = sum(len(f) for f in fd['features_per_frame'])
print(f"Detected {total_feats} features across all frames.")

# Inspect the frames 0-3 by changing frame_idx 
frame_idx = 0

im = plt.imshow(fd['labeled_masks'][frame_idx], cmap='plasma')
plt.title(f'Detected Features Frame {frame_idx}')




### 4.2 🔗 Particle Tracking

Particle tracking uses nearest neighbours to track the position of particles across frames. These 'tracks' are output in the 'tracks' dictionary of the particle tracking output dictionary.  


In [ ]:

tracks_out = stack.analysis['step_2_particle_tracking']['tracks']
print(f"Constructed {len(tracks_out)} tracks.")




### 4.3 🗃️ Flatten Tracks into Table

Within the analysis package, there are various utility functions to help with the processing of analysis output data. These are found in the `untils` subpackage which has helper functions for various analyses. Those built for particle analysis and tracking are in the `particles` module within `analysis.utils`.

The `flatten_particle_features` function in this module can be used to combine the results of particle detection modules with the corresponding results of a particle grouping (tracking or clustering) module. This means all the data and statistics measured when the particles were detected.


In [ ]:

tab = flatten_particle_features(
    grouping_output=stack.analysis['step_2_particle_tracking'],
    detection_output=fd,
    object_key='tracks',
    object_id_field='track_id'
)
print(f"Flattened table has {len(tab)} rows.")
tab.head()




### 4.4 📈 Visualising Tracks in 3D

Once tracked and in a table, various feature or particle metrics can be plotted per particle over time. A simple example is plotting the centroid position per track in 3D, with the centroid coordinates as the X and Y values and the detection's timestamp as the Z value. This is enabled by the `plot_particle_labels_3d` in the `playNano.analysis.utils.particles' module.


In [ ]:

fig = plot_particle_labels_3d(tab, object_id_field='track_id')
plt.show()




---

## 5. 💾 Exporting Results

### 5.1 📦 Processed Data Bundles

In addition to being able to save processed data as GIF animations for presentation, the raw and processed 3D numpy arrays can be exported as either a TIF files containing a stack of frames along with metadata compliant with the OME-TIF format (.ome.tif), a numpy zippied archive containing the array and metadata (.npz) or an HDF5 container with same data (.h5). 

There are functions within the `playNano.io.export` module for these operations.


In [ ]:
out = Path('output')
base = 'demo'
save_ome_tiff_stack(out/f"{base}.ome.tif", 
                    stack,
                    raw=False)
save_npz_bundle(out/base,
                    stack,
                    raw=False)
save_h5_bundle(out/base, 
               stack,
               raw=False)



### 5.2 📊 Analysis Exports

The results from the analysis modules can be exported to a JSON file using the `export_analysis_to_json` function within `playNano.analysis.utils.common` and the df generated by `flatten_particle_features` can be simply saved as a CSV file with the `.to_csv()` pandas method.


In [ ]:

export_analysis_to_json(out/f"{base}_analysis.json", analysis_record)
csv_path = out/f"{base}_tracks.csv"
tab.to_csv(csv_path, index=False)




**Next Steps:** Try different thresholds in `feature_detection`, adjust `max_distance` in `particle_tracking`, or experiment with `log_blob_detection` as an alternative feature detector. **Enjoy exploring your time-series AFM data!**
